In [1]:
import os
import numpy as np
from utils import load_seizure, split_data
from datasetConstruct import CustomDataset
from torch.utils.data import DataLoader
from models import CNN1D, train_using_optimizer, Wavenet, LSTM, evaluate_model

data_folder = "data"
MODEL_FOLDER = "D:/Blcdata/seizure/Model"
PAT_NO = 66

In [2]:
data_folder = os.path.join(data_folder, f"P{PAT_NO}")
seizure = load_seizure(data_folder, 1)

In [3]:
# Divide the seizure and nonseizure data in to 1s windows
window_size = 1
fs = seizure.samplingRate

seizure_data = seizure.ictal
nonseizure_data = seizure.interictal
nonseizure_data_postictal = seizure.postictal

seizure_data = split_data(seizure_data, fs)
nonseizure_data = split_data(nonseizure_data, fs)
nonseizure_data_postictal = split_data(nonseizure_data_postictal, fs)


In [4]:
# Combine the nonseizure and postictal data
nonseizure_data = np.concatenate((nonseizure_data, nonseizure_data_postictal), axis=0)

# Create the labels
seizure_labels = np.ones(len(seizure_data))
nonseizure_labels = np.zeros(len(nonseizure_data))

seizure_data = seizure_data.transpose(0, 2, 1)
nonseizure_data = nonseizure_data.transpose(0, 2, 1)

channels = seizure_data.shape[1]
time_steps = seizure_data.shape[2]

# Combine the dataset and labels, then shuffle them and create training and validation sets
data = np.concatenate((seizure_data, nonseizure_data), axis=0)
labels = np.concatenate((seizure_labels, nonseizure_labels), axis=0)

# Shuffle the data
shuffled_indices = np.random.permutation(len(data))
data = data[shuffled_indices]

labels = labels[shuffled_indices]

# Create the training and validation sets
train_data = data[:int(0.8 * len(data))]
train_labels = labels[:int(0.8 * len(labels))]
val_data = data[int(0.8 * len(data)):]
val_labels = labels[int(0.8 * len(labels)):]


In [5]:
# Load the dataset
train_dataset = CustomDataset(train_data, train_labels)
val_dataset = CustomDataset(val_data, val_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)


In [6]:
# Create the model

model1 = CNN1D(input_dim=channels, kernel_size=time_steps, output_dim=2)
model2 = Wavenet(input_dim=channels, output_dim=2, kernel_size=time_steps)
model3 = LSTM(input_dim=channels, output_dim=2)

# Train the model
CNNtrain_loss, CNNval_los, CNNval_accuracy = train_using_optimizer(model1, train_loader, val_loader, MODEL_FOLDER)
Wavetrain_loss, Waveval_los, Waveval_accuracy = train_using_optimizer(model2, train_loader, val_loader, MODEL_FOLDER)

# Evaluate the model
loss_CNN, acuracy_CNN = evaluate_model(model1, val_loader,'cuda:0')
# loss_LSTM, acuracy_LSTM = evaluate_model(model3, val_loader,'cuda:0')
loss_Wavenet, acuracy_Wavenet = evaluate_model(model2, val_loader,'cuda:0')

C:\Users\arthu\anaconda3\envs\Code\lib\site-packages\torch\nn\modules\conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ..\aten\src\ATen\native\Convolution.cpp:1004.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch [1/200], Loss: 1.0894
Epoch [2/200], Loss: 1.1079
Epoch [3/200], Loss: 1.1012
Epoch [4/200], Loss: 1.1012
Epoch [5/200], Loss: 1.1079
Epoch [6/200], Loss: 1.0945
Epoch [7/200], Loss: 1.1079
Epoch [8/200], Loss: 1.0811
Epoch [9/200], Loss: 1.0878
Epoch [10/200], Loss: 1.0945
Epoch [11/200], Loss: 1.0878
Epoch [12/200], Loss: 1.0945
Epoch [13/200], Loss: 1.0945
Epoch [14/200], Loss: 1.0811
Epoch [15/200], Loss: 1.1079
Epoch [16/200], Loss: 1.1012
Epoch [17/200], Loss: 1.0945
Epoch [18/200], Loss: 1.0945
Epoch [19/200], Loss: 1.0945
Epoch [20/200], Loss: 1.0878
Validation Loss: 0.0417, Validation Accuracy: 0.2703
Epoch [21/200], Loss: 1.0811
Epoch [22/200], Loss: 1.1079
Epoch [23/200], Loss: 1.1012
Epoch [24/200], Loss: 1.1079
Epoch [25/200], Loss: 1.1012
Epoch [26/200], Loss: 1.0945
Epoch [27/200], Loss: 1.0945
Epoch [28/200], Loss: 1.1012
Epoch [29/200], Loss: 1.0945
Epoch [30/200], Loss: 1.0945
Epoch [31/200], Loss: 1.1012
Epoch [32/200], Loss: 1.1012
Epoch [33/200], Loss: 1.0878